# Lab 15: Differential Photometry and Lightcurves
This lab will demonstrate how to go from a series of aperture photometry tables to a lightcurve.

In [ ]:
# initial imports
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import astropy.units as u

# run the %matplotlib magic command to enable inline plotting
# in the current Notebook
%matplotlib inline

## Lightcurves
One of the most fundamental tools to undertanding a variable object whether it be a variable star, transiting exoplanet, or a supernova is to look at its lightcurve. Time Series Data is data taken over a multiple measurements (or epochs) with the expectation that the measurment will change in time. All time series data has the same general form: 

Date1 Measurement1 Measurement1_error<br /> 
Date2 Measurement2 Measurement2_error<br />
Date3 Measurement3 Measurement3_error<br />
etc

The dates are almost always in a Julian Date format, and these days most people use either Heliocentric or Barycentric Julian Dates (HJD or BJD). For lightcurves the measurment is either in flux or magnitudes.

An Example lightcurve is below:

In [ ]:
lightcurve = Table.read('data/SW_And_V.cur',format='ascii',names=('JD','Vmag','Vmag_err'))
plt.plot(lightcurve['JD']-2400000.5,lightcurve['Vmag'],'o')
plt.xlabel('MJD') #Plotting an MJD just to avoid large numbers.
plt.ylabel('V Magnitude')
plt.gca().invert_yaxis() #Magnitudes are upside down
plt.show()

## Differential Photometry
There are many different ways to measure flux or magnitudes but the one we are most interested in for our measurements is differential photometry. The idea behind differential photometry is that if you take an image of multiple stars, then all those stars are going to be affected by the same outside factors (e.g. transparency, seeing, clouds etc.). By comparing our target star to comparison stars in the same frame, we can remove many of these outside factors.

There are different methods of doing differential photometry, the one we will use is called relative flux. Let's say that we a target star T, and three comparison stars A, B, and C. $T_{flux}$ will just be the sky subtracted flux in electrons from the target. Likewise for $A_{flux}$, $B_{flux}$, and $C_{flux}$. Our relative flux of, $T_{rel}$ is then:

$T_{rel} = \frac{T_{flux}}{A_{flux} + B_{flux} + C_{flux}}$ 

If we want, we can normalize the flux by dividing each $T_{rel}$ by the first value **Fix Nomenclature for $T1_{rel}$ the constant (denominator)**

$T1_{norm} = \frac{T1_{rel}}{T1_{rel}}$ <br />
$T2_{norm} = \frac{T2_{rel}}{T1_{rel}}$ <br />
$T3_{norm} = \frac{T3_{rel}}{T1_{rel}}$ <br />
etc.

If none of the stars are variable then the normalized fluxes will all equal 1.

We can also find the error in the relative flux, $T1_{rel,err}$ by using standard error propagation. Let $T1_{err}$, $A1_{err}$, $B1_{err}$, and $C1_{err}$ be the sky subtraced errors from you apeture photometry. Then:

$T1_{rel,err} = \sqrt{ T1_{rel}^2 \times \left(\frac{T1_{err}^2}{T1_{flux}^2} + \frac{A1_{err}^2 + B1_{err}^2 + C1_{err}^2}{(A1_{flux} + B1_{flux} + C1_{flux})^2} \right)}$ 

The normalized error, $T1_{norm,err}$, is just 

$T1_{norm,err} = \frac{T1_{rel,err}}{T1_{rel}}$ <br />
$T2_{norm,err} = \frac{T2_{rel,err}}{T1_{rel}}$ <br />
$T3_{norm,err} = \frac{T3_{rel,err}}{T1_{rel}}$ <br />
etc.

since $T1_{rel}$ is a normalization constant in this situation and thus has no error associated with it.

# Load in Photometry Data
I have saved my apeture photometry into a csv file with the colums $HJD$, $T_{flux}$, $T_{err}$, $A_{flux}$, $A_{err}$, $B_{flux}$, $B_{err}$, $C_{flux}$, $C_{err}$ 

In [ ]:
phot_tab = Table.read('data/AV_Peg24_V.csv')
phot_tab

## Determine my normalized flux

In [ ]:
t_rel = phot_tab['Source-Sky_T']/ (phot_tab['Source-Sky_A'] +phot_tab['Source-Sky_B']+phot_tab['Source-Sky_C'])
t_rel

In [ ]:
t_norm = t_rel / t_rel[0]
t_norm

In [ ]:
t_rel_err = np.sqrt( t_rel**2 * ((phot_tab['Source_Error_T']/phot_tab['Source-Sky_T'])**2 + (
    phot_tab['Source_Error_A']**2 + phot_tab['Source_Error_B']**2+ phot_tab['Source_Error_C']**2)/
                    (phot_tab['Source-Sky_A'] + phot_tab['Source-Sky_B']+ phot_tab['Source-Sky_C'])**2))
t_rel_err

In [ ]:
t_norm_err = t_rel_err / t_rel[0]
t_norm_err

## Create my Lightcurve

In [ ]:
lightcurve_tab = Table()
lightcurve_tab['HJD'] = phot_tab['HJD']
lightcurve_tab['Flux'] = t_norm
lightcurve_tab['Flux_Err'] = t_norm_err
lightcurve_tab

## Plot the lightcurve

In [ ]:
plt.errorbar(lightcurve_tab['HJD'] - 2454313
             ,lightcurve_tab['Flux'],yerr=lightcurve_tab['Flux_Err'],fmt='b.',label="Data")
plt.ylabel('Normalized Flux')
plt.xlabel("HJD - 2454313")
plt.gca().axvline(2454313.756 - 2454313,linestyle="--",label='Maximum Light') #Plot line at the maximum 
plt.legend()
plt.show()

## Check your Calibrations
It is always good to check that your calibration stars look good. You may find that one of them is not as good as the others, and then you may want to use a different star.

In [ ]:
C_A = phot_tab['Source-Sky_A']/ (phot_tab['Source-Sky_B']+phot_tab['Source-Sky_C'])
C_A_norm  = C_A / C_A[0]
C_B = phot_tab['Source-Sky_B']/ (phot_tab['Source-Sky_A']+phot_tab['Source-Sky_C'])
C_B_norm  = C_B / C_B[0]
C_C = phot_tab['Source-Sky_C']/ (phot_tab['Source-Sky_A']+phot_tab['Source-Sky_B'])
C_C_norm  = C_C / C_C[0]

In [ ]:
plt.plot(lightcurve_tab['HJD'] - 2454313,C_A_norm,'.',label="Calib A") 
plt.plot(lightcurve_tab['HJD'] - 2454313,C_B_norm -.2,'.',label="Calib B") #Adding small shift to separate on Graph
plt.plot(lightcurve_tab['HJD'] - 2454313,C_C_norm -.4,'.',label="Calib C") #Adding small shift to separate on Graph
plt.ylabel('Normalized Flux')
plt.xlabel("HJD - 2454313")
plt.ylim(0.3,1.1)
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

Clearly my C calibration star is significantly worse than my A and B stars. In this case, it might not be bad enough to throw it out, but it might be worth looking for another calibration star.